In [ ]:
!pip install python-dotenv

In [ ]:
!pip list

In [ ]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
os.getenv('X-API-Key')

In [ ]:
def fetch_data(url, api_key):
    headers = {'X-API-Key': api_key}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # This will raise an exception for HTTP errors

    total_pages = response.headers.get('X-Total-Pages', 1)  # Default to 1 if the header is not found
    return response.json(), int(total_pages)

def extract_payload_and_time(data):
    return [(entry['payload'], entry['rx_metadata'][0].get('received_at')) for entry in data]


def main():
    base_url = os.getenv('BASE_URL')
    api_key = os.getenv('X-API-Key')      
    all_data = []
    
    # Fetch data from the first page to get the total number of pages
    first_page_url = base_url + '1'
    first_page_data, total_pages = fetch_data(first_page_url, api_key)
    all_data.extend(extract_payload_and_time(first_page_data))

    # Fetch data from the remaining pages
    for page in range(2, total_pages + 1):
        url = base_url + str(page)
        data, _ = fetch_data(url, api_key)
        all_data.extend(extract_payload_and_time(data))

    return all_data

In [ ]:
all_data = main()
all_data

In [ ]:
import json

In [ ]:
# Specify the filename
filename = 'globalsat_lt501e.json'

# Writing to a JSON file
with open(filename, 'w') as file:
    json.dump(all_data, file, indent=4)

print(f"Data has been exported to {filename}")

In [ ]:
all_data[0][1]

In [ ]:
import base64

# Example base64 encoded string
encoded_value = all_data[0][0]

# Decode the base64 encoded string
decoded_value = base64.b64decode(encoded_value)
decoded_value.hex()

In [ ]:
from parser.parser_globalsat_lt501e import LT501Parser

parser = LT501Parser(debug_level=99)
output = parser.parse_bytes_hex(decoded_value.hex())
output
print(output)
print(output['lat'])

In [ ]:
import base64
from parser.parser_globalsat_lt501e import LT501Parser

# Initialize the parser
parser = LT501Parser(debug_level=99)

# List to hold the parsed data
parsed_data = []

for encoded_value, time in all_data:
    # Decode the base64 encoded string
    decoded_value = base64.b64decode(encoded_value)

    # Use the parser to parse the decoded data
    parsed_entry = parser.parse_bytes_hex(decoded_value.hex())
    if 'lon' in parsed_entry and 'lat' in parsed_entry:
        
        # Append the parsed data in the required format
        parsed_data.append({
            'coordinates': [parsed_entry['lon'], parsed_entry['lat']],
            'time': time
        })
    else:
        print(f"Missing 'lon' or 'lat' in entry: {parsed_entry}")
        continue    
    
parsed_data

In [ ]:
from map.plot_gps_history import create_geojson_features
import folium
from folium.plugins import TimestampedGeoJson

# Create map object
m = folium.Map(location=parsed_data[0]['coordinates'], zoom_start=3)

# Create features
features = create_geojson_features(parsed_data)

# Create a TimestampedGeoJson object and add it to the map
TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M',
    transition_time=1000,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY/MM/DD HH:mm:ss',
    time_slider_drag_update=True,
    add_last_point=True).add_to(m)

# Save to an HTML file
m.save('my_animated_map.html')

print("Map has been saved to my_animated_map.html")
